## 連立一次方程式を Python の数理最適化 Libray で解く

In [7]:
import pandas as pd
import pulp  # PuLP library の取り込み

problem = pulp.LpProblem('SLE', pulp.LpMaximize)  # 数理 Model の定義。第１引数は任意の名前。第２引数は、最大化問題を解く、という指定（※今回は最大化問題を解いているわけではないのでおまじないに近い）

x = pulp.LpVariable('x', cat='Continuous')  # 変数 x を定義。pulp.LaVariable('任意の名前', cat='変数が連続変数、つまり実現値をとることを指定')
y = pulp.LpVariable('y', cat='Continuous')  # 変数 y を定義。pulp.LaVariable('任意の名前', cat='変数が連続変数、つまり実現値をとることを指定')

problem += 120 * x + 150 * y == 1440  # 連立一次方程式を定義
problem += x + y == 10  # 連立一次方程式を定義

status = problem.solve()  # 定義した数理 Model を解く。解けたか解けなかったの情報を返す

print('Status:', pulp.LpStatus[status])  # 最適化計算をした結果を表示
print('x=', x.value(), 'y=', y.value())  # x, y 計算結果の値を表示

Status: Optimal
x= 2.0 y= 8.0


## 線形化問題を Python の数理最適化 Libray で解く

ある工場では製品 *p* と *q* を製造しています。製品 *p* と *q* を製造するには原料 *m* と *n* が必要で、次のことがわかっています。
- 製品 *p* を 1kg 製造するには原料 *m* が 1kg、原料 *n* が 2kg 必要である。
- 製品 *q* を 1kg 製造するには原料 *m* が 3kg、原料 *n* が 1kg 必要である。
- 原料 *m* の在庫は 30kg、原料 *n* の在庫は 40kg である。
- 単位量あたりの利得は、製品 *p* は１万円、製品 *q* は２万円である。

この条件で利得を最大にするためには、製品 *p* と *q* をそれぞれ何 kg 製造すればよいでしょうか。

In [8]:
problem = pulp.LpProblem('LP', pulp.LpMaximize)  # PuLP の取り込み

x = pulp.LpVariable('x', cat='Continuous')  # 変数x をpulp.LpVariable('任意の名前', cat='連続変数、つまり実現値をとることを指定')
y = pulp.LpVariable('y', cat='Continuous')  # 変数y をpulp.LpVariable('任意の名前', cat='連続変数、つまり実現値をとることを指定')

problem += 1*x + 3*y <= 30  # 線形問題の条件式を定義
problem += 2*x + 1*y <= 40  # 線形問題の条件式を定義
problem += x >= 0  # 線形問題の条件式を定義
problem += y >= 0  # 線形問題の条件式を定義

problem += x + 2*y  # 数理 Model: problem の目的関数を定義
# LpProblem() は、+= 演算子の右辺が制約であれば制約式を追加し、関数であれば目的関数を追加する

status = problem.solve()  # 定義した数理 Model を解く

print('Status:', pulp.LpStatus[status])  # 解けたか解けなかったか結果を表示
print('x=', x.value(), 'y=', y.value(), 'obj=', problem.objective.value())  # x, y と最大化しようとした利得の関数値を表示

Status: Optimal
x= 18.0 y= 4.0 obj= 26.0


線形計画課問題を解いた結果
- 製品 *p* を 18kg
- 製品 *q* を 4kg

生産すると利得が最大になることがわかった。

## 規模の大きな数理最適化問題を Python の Library で解く

### 【問題】
ある工場では製品 *p1*, *p2*, *p3*, *p4* の４種類を製造しています。
製品 *p1*, *p2*, *p3*, *p4* の製造には、原料 *m1*, *m2*, *m3*, の３種類が必要です。
また、その量は File「requires.csv」に記述されています。

In [9]:
!type requires.csv

p,m,require
p1,m1,2
p1,m2,0
p1,m3,1
p2,m1,3
p2,m2,2
p2,m3,0
p3,m1,0
p3,m2,2
p3,m3,2
p4,m1,2
p4,m2,2
p4,m3,2


本Data の２～４行目は、製品 *p1* を 1kg 製造するには原料 *m1* が 2kg, 原料 *m2* が 0kg、、原料 *m3* が1kg 必要であることを表しています。
また、原料  *m1*, *m2*, *m3* の在庫は stocks.csv に記述されています。

In [10]:
!type stocks.csv

m,stock
m1,35
m2,22
m3,27


本Data の２行目は、原料 *m1* の在庫は 35kg であることを表しています。
最後に、製品 *p1*, *p2*, *p3*, *p4* を精算した際の利得は gains.csv に記述されています。

In [11]:
!type gains.csv

p,gain
p1,3
p2,4
p3,4
p4,5


本Data の２行目の Data は、製品 *p1* の利得は、３万円であることを表しています。
このとき、利得を最大にするためには、製品 *p1*, *p2*, *p3*, *p4*, をそれぞれ何kg 製造すればよいでしょうか。

### この線形計画問題の数理 Modeling
- List
- 定数
- 変数

を準備する。そのうえで

- 制約式
- 目的関数

を定める。

数理 Model を関数として捉えると

| 要素       | 対応             |
|----------|----------------|
| List と定数 | 入力に対応。         |
| 変数       | 出力（決定したい対象）に対応 |
| 制約式と目的関数 | 関数定義に当たる       |

In [12]:
# 必要な Libray の import
import pandas as pd
import pulp
from IPython.display import display

In [13]:
# Data の 取得
stock_df = pd.read_csv('stocks.csv')
display(stock_df)
require_df = pd.read_csv('requires.csv')
display(require_df)
gain_df = pd.read_csv('gains.csv')
display(gain_df)

,m,stock
0,m1,35
1,m2,22
2,m3,27


,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [14]:
# List の定義
P = gain_df['p'].tolist()
print('P', P)
M = stock_df['m'].tolist()
print('M', M)

P ['p1', 'p2', 'p3', 'p4']
M ['m1', 'm2', 'm3']


#### List の定義
- *P*: 製品の List
- *M*: 原料の List

*P* は、gain_df から、 *M* は、stock_df から取得する

In [15]:
# List の定義
P = gain_df['p'].tolist()
print('P', P)
M = stock_df['m'].tolist()
print('M', M)

P ['p1', 'p2', 'p3', 'p4']
M ['m1', 'm2', 'm3']


#### 定数の定義
- *stock_m* (*m* ∊ *M*): 原料 *m* の在庫量
- *require_p,m* (*p* ∊ *P*, *m* ∊ *M*): 製品 *p* を生産するのに必要な原料 *m* の量
- *gain_p* (*p* ∊ *P*): 製品 *p* を生産した際の利得

それぞれ、stock_df, require_df, gain_df から取得。
dict型で作成する

In [16]:
# 定数の定義
stock = {row.m: row.stock for row in stock_df.itertuples()}
print('stock_m', stock)
require = {(row.p, row.m): row.require for row in require_df.itertuples()}
print('require_p,m', require)
gain = {row.p: row.gain for row in gain_df.itertuples()}
print('gain_p', gain)

stock_m {'m1': 35, 'm2': 22, 'm3': 27}
require_p,m {('p1', 'm1'): 2, ('p1', 'm2'): 0, ('p1', 'm3'): 1, ('p2', 'm1'): 3, ('p2', 'm2'): 2, ('p2', 'm3'): 0, ('p3', 'm1'): 0, ('p3', 'm2'): 2, ('p3', 'm3'): 2, ('p4', 'm1'): 2, ('p4', 'm2'): 2, ('p4', 'm3'): 2}
gain_p {'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}


#### 線形計画問題の定義
最大化問題として線形計画課問題を定義する

In [17]:
# 線形計画問題の定義
problem = pulp.LpProblem('LP2', pulp.LpMaximize)
# 第１引数は任意の名前で問題なし
# 第２引数に pulp.LpMaximize を指定することで最大化問題として定義できる

#### 変数の定義
*x_p(p∊P)* : 製品 *p* の生産量

In [18]:
# 変数の定義
x = pulp.LpVariable.dicts('x', P, cat='Continuous')  # pulp.LpVariable.dicts() で変数をまとめて定義できる。

#### 制約式の定義
- *x_p≧(p ∊ P)*: 製品 *P* の生産量は 0 以上
- *∑_p∊P require_p,m･x_p ≦ stock_m(m∊M)*: 生産は在庫の範囲で行なう

In [19]:
# 制約式の定義

# 各要素を for文でまわして１つずつ制約式を定義していく
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m]
    # pulp.lpSum() で総和記号 ∑ を表現できる

#### 目的関数の定義
目的関数（最大化）
*∑_p∊P gain_p･x_p*

In [20]:
# 目的関数の定義
problem += pulp.lpSum([gain[p] * x[p] for p in P])

In [21]:
# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

Status: Optimal


In [22]:
# 計算結果の表示
for p in P:
    print(p, x[p].value())

print('obj=', problem.objective.value())

p1 12.1429
p2 3.57143
p3 7.42857
p4 0.0
obj= 80.42869999999999


利得が最大になるような各製品の生産量と利得が確認できる。

## 規模の大きな数理最適化問題を Python の Library で解く

### 【問題】
ある工場では製品 *p1*, *p2*, *p3*, *p4* の４種類を製造しています。
製品 *p1*, *p2*, *p3*, *p4* の製造には、原料 *m1*, *m2*, *m3*, の３種類が必要です。
また、その量は File「requires.csv」に記述されています。

In [7]:
!type requires.csv

p,m,require
p1,m1,2
p1,m2,0
p1,m3,1
p2,m1,3
p2,m2,2
p2,m3,0
p3,m1,0
p3,m2,2
p3,m3,2
p4,m1,2
p4,m2,2
p4,m3,2


本Data の２～４行目は、製品 *p1* を 1kg 製造するには原料 *m1* が 2kg, 原料 *m2* が 0kg、、原料 *m3* が1kg 必要であることを表しています。
また、原料  *m1*, *m2*, *m3* の在庫は stocks.csv に記述されています。

In [9]:
!type stocks.csv

m,stock
m1,35
m2,22
m3,27


本Data の２行目は、原料 *m1* の在庫は 35kg であることを表しています。
最後に、製品 *p1*, *p2*, *p3*, *p4* を精算した際の利得は gains.csv に記述されています。

In [10]:
!type gains.csv

p,gain
p1,3
p2,4
p3,4
p4,5


本Data の２行目の Data は、製品 *p1* の利得は、３万円であることを表しています。
このとき、利得を最大にするためには、製品 *p1*, *p2*, *p3*, *p4*, をそれぞれ何kg 製造すればよいでしょうか。

### この線形計画問題の数理 Modeling
- List
- 定数
- 変数

を準備する。そのうえで

- 制約式
- 目的関数

を定める。

数理 Model を関数として捉えると
- List, 定数は、入力に対応。
- 変数は、出力（決定したい対象）に対応
- 制約式、- 目的関数は、関数定義に当たる

In [15]:
# 必要な Libray の import
import pandas as pd
import pulp
from IPython.display import display

In [54]:
# Data の 取得
stock_df = pd.read_csv('stocks.csv')
display(stock_df)
require_df = pd.read_csv('requires.csv')
display(require_df)
gain_df = pd.read_csv('gains.csv')
display(gain_df)

,m,stock
0,m1,35
1,m2,22
2,m3,27


,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [ ]:
# List の定義
P = gain_df['p'].tolist()
print('P', P)
M = stock_df['m'].tolist()
print('M', M)

#### List の定義
- *P*: 製品の List
- *M*: 原料の List

*P* は、gain_df から、 *M* は、stock_df から取得する

In [57]:
# List の定義
P = gain_df['p'].tolist()
print('P', P)
M = stock_df['m'].tolist()
print('M', M)

P ['p1', 'p2', 'p3', 'p4']
M ['m1', 'm2', 'm3']


#### 定数の定義
- *stock_m* (*m* ∊ *M*): 原料 *m* の在庫量
- *require_p,m* (*p* ∊ *P*, *m* ∊ *M*): 製品 *p* を生産するのに必要な原料 *m* の量
- *gain_p* (*p* ∊ *P*): 製品 *p* を生産した際の利得

それぞれ、stock_df, require_df, gain_df から取得。
dict型で作成する

In [59]:
# 定数の定義
stock = {row.m: row.stock for row in stock_df.itertuples()}
print('stock_m', stock)
require = {(row.p, row.m): row.require for row in require_df.itertuples()}
print('require_p,m', require)
gain = {row.p: row.gain for row in gain_df.itertuples()}
print('gain_p', gain)

stock_m {'m1': 35, 'm2': 22, 'm3': 27}
require_p,m {('p1', 'm1'): 2, ('p1', 'm2'): 0, ('p1', 'm3'): 1, ('p2', 'm1'): 3, ('p2', 'm2'): 2, ('p2', 'm3'): 0, ('p3', 'm1'): 0, ('p3', 'm2'): 2, ('p3', 'm3'): 2, ('p4', 'm1'): 2, ('p4', 'm2'): 2, ('p4', 'm3'): 2}
gain_p {'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}


In [60]:
# 線形計画問題の定義
problem = pulp.LpProblem('LP2', pulp.LpMaximize)

In [61]:
# 変数の定義
x = pulp.LpVariable.dicts('x', P, cat='Continuous')

In [62]:
# 制約式の定義
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m]

In [63]:
# 目的関数の定義
problem += pulp.lpSum([gain[p] * x[p] for p in P])

In [64]:
# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

Status: Optimal


In [65]:
# 計算結果の表示
for p in P:
    print(p, x[p].value())

print('obj=', problem.objective.value())

p1 12.1429
p2 3.57143
p3 7.42857
p4 0.0
obj= 80.42869999999999


In [3]:
!type requires.csv

p,m,require
p1,m1,2
p1,m2,0
p1,m3,1
p2,m1,3
p2,m2,2
p2,m3,0
p3,m1,0
p3,m2,2
p3,m3,2
p4,m1,2
p4,m2,2
p4,m3,2


本Data の２～４行目は、製品 *p1* を 1kg 製造するには原料 *m1* が 2kg, 原料 *m2* が 0kg、、原料 *m3* が1kg 必要であることを表しています。
また、原料  *m1*, *m2*, *m3* の在庫は stock.csv に記述されています。